# Bloch-Torrey Equation

## Introduction

Here we solve the Bloch-Torrey equation on a unit square, with the diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$ all given as a generic functions.
The strong form of the Bloch-Torrey equation is given by

\begin{align}
    \frac{\partial u_x}{\partial t} &= \nabla \cdot (D \nabla u_x) - R u_x + \omega u_y  \quad x \in \Omega\\
    \frac{\partial u_y}{\partial t} &= \nabla \cdot (D \nabla u_y) - R u_y - \omega u_x  \quad x \in \Omega,
\end{align}

where $\vec{u}=[u_x,u_y]$ is the transverse magnetization, and $\Omega$ the domain.

We will consider homogeneous Neumann boundary conditions such that

\begin{align}
    \nabla \vec{u}(x) \cdot \hat{n} &= \vec{0}  \quad x \in \partial \Omega\\
\end{align}

where $\partial \Omega$ denotes the boundary of $\Omega$, and $\cdot$ is a tensor contraction.

The initial condition is given generically as

\begin{equation}
    \vec{u}(x,t=0) = \vec{u}_0 (x)  \quad x \in \Omega
\end{equation}


The resulting weak form is given by
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    &= -\int_{\Omega}
    -\vec{v} \cdot \nabla \cdot ( D \, \nabla \vec{u} ) +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega \\
    &= -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega + 
    \int_{\partial\Omega} \vec{v} \cdot (D\nabla\vec{u} \cdot \hat{n}) \, d\Gamma,
\end{align}
where $\vec{v}$ is a suitable test function.

In this notebook, we will assume homogeneous Neumann boundary conditions on all boundaries by taking $D\nabla\vec{u} \cdot \hat{n} = 0$. Therefore, the final weak form is simply
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    = -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega
\end{align}

Note: in two dimensions, the cross product is simply a scalar. However, `Tensors.jl` defines the two dimensional cross product by first extending the 2D vectors into 3D. Below, we use the symbol $\boxtimes$ to denote the scalar version, which is the same as taking the third component of the vector version

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [ ]:
# using Distributed
# addprocs(8; restrict = true, enable_threaded_blas = true);
# @everywhere begin
#     include("init.jl")
# end

In [ ]:
include("init.jl")

**Pack circles**: Pack circles with a specified packing density $\eta$

In [ ]:
btparams = BlochTorreyParameters{Float64}(theta = pi/2, g_ratio = 0.8);

In [ ]:
Dim = 2;
Ncircles = 20;

rs = rand(radiidistribution(btparams), Ncircles);
initial_circles = GreedyCirclePacking.pack(rs; iters = 100);

η = btparams.g_ratio; # goal packing density
ϵ = 0.1 * btparams.R_mu; # overlap occurs when distance between circle edges is ≤ ϵ
α = 1e-1; # covariance penalty weight (enforces circular distribution)
β = 1e-6; # mutual distance penalty weight
λ = 1.0; # overlap penalty weight (or lagrange multiplier for constrained version)
w = [α, β, λ]; # vector of weights

In [5]:
minimum_signed_edge_distance(initial_circles)

-4.996003610813204e-16

In [6]:
estimate_density(initial_circles)

0.798012578855469

In [7]:
@time outer_circles = EnergyCirclePacking.pack(initial_circles;
    autodiff = false,
    secondorder = false,
    setcallback = false,
    goaldensity = η,
    distancescale = btparams.R_mu,
    weights = w,
    epsilon = ϵ);

inner_circles = scale_shape.(outer_circles, btparams.g_ratio);

  7.779167 seconds (14.68 M allocations: 729.247 MiB, 11.95% gc time)


In [8]:
dmin = minimum_signed_edge_distance(outer_circles)
@show covariance_energy(outer_circles)
@show estimate_density(outer_circles)
@show is_any_overlapping(outer_circles)
@show (dmin, ϵ, dmin > ϵ);

covariance_energy(outer_circles) = 0.004383245229828704
estimate_density(outer_circles) = 0.8
is_any_overlapping(outer_circles) = false
(dmin, ϵ, dmin > ϵ) = (0.047259641468211766, 0.046000000000000006, true)


**Generate mesh**: Rectangular mesh with circles possibly only partly contained or completely excluded

In [9]:
#Ncircles = length(outer_circles);
#Nmin = 50; # points for average circle
#h0 = 2pi*R_mu/Nmin; # approximate scale
h0 = minimum(radius.(outer_circles))*(1-btparams.g_ratio); # fraction of size of minimum torus width
# eta = 5.0; # approx ratio between largest/smallest edges, i.e. max ≈ eta * h0
# gamma = 10.0; # max edge length of `eta * h0` occurs approx. `gamma * h0` from circle edges
# alpha = 0.7; # power law for edge length scaling between `h0` and `eta * h0`
h_min = 1.0h0; # minimum edge length
h_max = 5.0h0; # maximum edge length
h_range = 10.0h0; # distance over which h increases from h_min to h_max
h_rate = 0.6; # rate of increase of h from circle boundaries (power law; smaller = faster radial increase)

0.6

In [10]:
bdry, _ = opt_subdomain(outer_circles)

(Rectangle{2,Float64}([-0.531818, -2.90496], [2.51431, 0.141174]), 0.75)

In [11]:
@time exteriorgrids, torigrids, interiorgrids, parentcircleindices = disjoint_rect_mesh_with_tori(
    bdry, inner_circles, outer_circles, h_min, h_max, h_range, h_rate;
    maxstalliters = 500, plotgrids = false, exterior_tiling = (2, 2)
);

1/20: Interior
1/20: Annular
2/20: Interior
2/20: Annular
3/20: Interior
3/20: Annular
4/20: Interior
4/20: Annular
5/20: Interior
5/20: Annular
6/20: Interior
6/20: Annular
7/20: Interior
7/20: Annular
8/20: Interior
8/20: Annular
9/20: Interior
9/20: Annular
10/20: Interior
10/20: Annular
11/20: Interior
11/20: Annular
12/20: Interior
12/20: Annular
13/20: Interior
13/20: Annular
14/20: Interior
14/20: Annular
15/20: Interior
15/20: Annular
16/20: Interior
16/20: Annular
17/20: Interior
17/20: Annular
18/20: Interior
18/20: Annular
19/20: Interior
19/20: Annular
20/20: Interior
20/20: Annular
1/4: Exterior
2/4: Exterior
3/4: Exterior
4/4: Exterior
 41.999336 seconds (198.11 M allocations: 6.684 GiB, 15.25% gc time)


In [12]:
mxcall(:figure, 0)
simpplot.(exteriorgrids; hold = true, axis = mxaxis(bdry));
simpplot.(interiorgrids; hold = true, axis = mxaxis(bdry));
simpplot.(torigrids; hold = true, axis = mxaxis(bdry));

### Diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$

These functions are defined within `doassemble!`

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [13]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions are handled by a `ConstraintHandler`. However, here we will have no need to directly enforce boundary conditions, since Neumann boundary conditions have already been applied in the derivation of the weak form.

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

In [14]:
# domains = MyelinDomain(grid, outer_circles, inner_circles, bdry, exteriorgrid, torigrids, interiorgrids; quadorder = 3, funcinterporder = 1);
myelindomains = createmyelindomains(exteriorgrids, torigrids, interiorgrids, outer_circles, inner_circles);

In [15]:
myelinprob = MyelinProblem(btparams);

In [16]:
# @time doassemble!(prob, domains);
@time map!(m -> doassemble!(m, myelinprob), myelindomains, myelindomains);

  3.561739 seconds (21.05 M allocations: 543.133 MiB, 12.56% gc time)


In [17]:
# @time factorize!(domains);
@time map!(m -> (factorize!(getdomain(m)); return m), myelindomains, myelindomains);

  0.350986 seconds (516.33 k allocations: 33.656 MiB, 7.08% gc time)


In [18]:
omegavalues = map(myelindomains) do m
    ω = BlochTorreyProblem(myelinprob, m).Omega
    return map(getnodes(getgrid(m))) do node
        ω(getcoordinates(node))
    end
end;

In [19]:
mxcall(:figure, 0)
for (m,w) in Iterators.reverse(zip(myelindomains, omegavalues))
    simpplot(getgrid(m); hold = true, axis = mxaxis(bdry), facecol = w);
end

┌ Warning: Deprecated syntax `implicit assignment to global variable `w``.
│ Use `global w` instead.
└ @ nothing none:0
┌ Warning: Loop variable `w` around In[19]:2 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[19]:2


### Solution of the differential equation system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [20]:
tspan = (0.0, 320.0e-3);
dt = 10e-3;
# saveat = tspan[1]:dt:tspan[2];
# tstops = (tspan[1] .+ dt/2 .+ dt .* (1:round(Int, (tspan[2]-tspan[1])/dt)))
u0 = Vec{2}((0.0, 1.0)); # initial pi/2 pulse

In [21]:
probs = [ODEProblem(m, interpolate(u0, m), tspan) for m in myelindomains];

In [22]:
sols = Vector{ODESolution}(undef, length(probs));

In [23]:
@time for i in eachindex(sols, probs)
    print("i = $i/$(length(sols)): ")
    sols[i] = @time solve(probs[i], ExpokitExpmv();
        reltol = 1e-4,
        dt = dt,
        m = 30,
        verbose = false,
        affect! = u -> (@views(u[2:end] .= -u[2:end]); return u) # spin echo pi-pulse
    )
end;

i = 1/39:  39.317342 seconds (46.94 M allocations: 6.160 GiB, 9.56% gc time)
i = 2/39:  13.803936 seconds (2.58 M allocations: 3.546 GiB, 4.75% gc time)
i = 3/39:   1.369554 seconds (299.96 k allocations: 352.873 MiB, 4.79% gc time)
i = 4/39:   4.811580 seconds (961.49 k allocations: 1.225 GiB, 4.47% gc time)
i = 5/39:   5.020810 seconds (935.59 k allocations: 1.217 GiB, 4.24% gc time)
i = 6/39:   4.267549 seconds (833.66 k allocations: 1.052 GiB, 4.56% gc time)
i = 7/39:  10.546062 seconds (1.96 M allocations: 2.734 GiB, 4.50% gc time)
i = 8/39:  13.094946 seconds (2.09 M allocations: 3.364 GiB, 4.36% gc time)
i = 9/39:  20.948107 seconds (2.76 M allocations: 5.388 GiB, 4.31% gc time)
i = 10/39:   1.359733 seconds (289.65 k allocations: 342.827 MiB, 4.90% gc time)
i = 11/39:  15.657121 seconds (2.68 M allocations: 4.243 GiB, 4.80% gc time)
i = 12/39:   7.040627 seconds (1.34 M allocations: 1.842 GiB, 4.79% gc time)
i = 13/39:   1.374340 seconds (305.69 k allocations: 351.450 MiB, 4.43

┌ Warning: `eye(T::Type, m::Integer)` has been deprecated in favor of `I` and `Matrix` constructors. For a direct replacement, consider `Matrix{T}(I, m, m)`. If `T` element type is not necessary, consider the shorter `Matrix(I, m, m)`(with default `eltype(I)` `Bool`)
│   caller = normest1(::LinearMaps.TransposeMap{Float64,ParabolicLinearMap{Float64}}, ::Int64, ::Array{Float64,2}) at Normest1.jl:101
└ @ Normest1 /home/coopar7/Documents/code/BlochTorreyExperiments-master/Experiments/MyelinWaterOrientation/Utils/Normest1.jl:101


 9.356654 seconds (1.09 M allocations: 2.465 GiB, 4.88% gc time)
i = 19/39:   9.192117 seconds (1.10 M allocations: 2.600 GiB, 5.02% gc time)
i = 20/39:   2.703179 seconds (490.70 k allocations: 648.618 MiB, 16.02% gc time)
i = 21/39:   3.110507 seconds (593.37 k allocations: 853.419 MiB, 5.11% gc time)
i = 22/39:   3.284599 seconds (608.83 k allocations: 889.624 MiB, 4.71% gc time)
i = 23/39:   2.926835 seconds (567.08 k allocations: 789.834 MiB, 4.32% gc time)
i = 24/39:   5.795130 seconds (902.49 k allocations: 1.604 GiB, 4.75% gc time)
i = 25/39:   6.255826 seconds (806.10 k allocations: 1.650 GiB, 4.54% gc time)
i = 26/39:   9.205235 seconds (1.14 M allocations: 2.659 GiB, 4.98% gc time)
i = 27/39:   2.448633 seconds (498.39 k allocations: 658.223 MiB, 4.86% gc time)
i = 28/39:   6.198561 seconds (943.96 k allocations: 1.787 GiB, 4.98% gc time)
i = 29/39:   6.357213 seconds (905.75 k allocations: 1.618 GiB, 4.04% gc time)
i = 30/39:   2.814025 seconds (562.02 k allocations: 772.21

In [24]:
S = map(myelindomains, sols) do m, s
    [integrate(u, m) for u in s.u]
end;

In [25]:
Stotal = sum(S);

In [26]:
Stotals = Dict{Int,Any}()
push!(Stotals, 90 => Stotal)
# push!(Stotals, 0 => Stotal)

Dict{Int64,Any} with 1 entry:
  90 => Tensor{1,2,Float64,2}[[0.0, 9.27558], [-0.157247, -6.8943], [0.620057, …

In [27]:
Tspan = tspan[2] - tspan[1]
@show btparams.R2_lp;
@show (-1/Tspan)*log(norm(Stotal[end])/norm(Stotal[1]));
@show exp(-tspan[end]*btparams.R2_lp);
@show norm(Stotal[end])/norm(Stotal[1]);

btparams.R2_lp = 15.873015873015873
(-1 / Tspan) * log(norm(Stotal[end]) / norm(Stotal[1])) = 18.583894413856722
exp(-(tspan[end]) * btparams.R2_lp) = 0.006223859418487457
norm(Stotal[end]) / norm(Stotal[1]) = 0.002614071758803262


In [28]:
myelin_area = intersect_area(outer_circles, bdry) - intersect_area(inner_circles, bdry)
total_area = area(bdry)
exact_mwf = myelin_area/total_area

0.2728252969910506

In [29]:
mxcall(:addpath, 0, "/home/coopar7/Documents/code/BlochTorreyExperiments-master/Experiments/MyelinWaterOrientation/MATLAB/SE_corr/");

In [30]:
MWImaps, MWIdist, MWIpart = fitmwfmodel(Stotal, NNLSRegression();
    T2Range = [10e-3, 2.0],
    spwin = [10e-3, 40e-3],
    nT2 = 32,
    PLOTDIST = true
);
getmwf(NNLSRegression(), MWImaps, MWIdist, MWIpart)

0.2913481146081364

In [31]:
# TestStotalMagn = mwimodel(TwoPoolMagnToMagn(), tspan[1]:dt:tspan[2], modelfit.param)
# TestStotal = [Vec{2}((zero(y),y)) for y in TestStotalMagn];

In [32]:
modeltypes = (TwoPoolMagnToMagn(), ThreePoolMagnToMagn(), ThreePoolCplxToMagn(), ThreePoolCplxToCplx())
for modeltype in modeltypes
    local modelfit, errors, mwf
    println("Model: $modeltype"); flush(stdout)
    modelfit, errors = fitmwfmodel(Stotal, modeltype; TE = dt);
    mwf = getmwf(modeltype, modelfit, errors)
    println("mwf: $mwf"); flush(stdout)
    #errors == nothing ? display(modelfit.param) : display([modelfit.param errors]); flush(stdout)
end

Model: TwoPoolMagnToMagn()
mwf: 0.5169445222367843
Model: ThreePoolMagnToMagn()
mwf: 0.5106716912493771
Model: ThreePoolCplxToMagn()
mwf: 0.22238069551780723
Model: ThreePoolCplxToCplx()
mwf: 0.9254125856070765


In [33]:
@show getmwf(Stotal, TwoPoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolMagnToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToMagn(); TE = dt, fitmethod = :local);
@show getmwf(Stotal, ThreePoolCplxToCplx(); TE = dt, fitmethod = :local);

getmwf(Stotal, TwoPoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.5169445222367843
getmwf(Stotal, ThreePoolMagnToMagn(); TE=dt, fitmethod=:local) = 0.5106716912493771
getmwf(Stotal, ThreePoolCplxToMagn(); TE=dt, fitmethod=:local) = 0.22238069551780723
getmwf(Stotal, ThreePoolCplxToCplx(); TE=dt, fitmethod=:local) = 0.9254125856070765


In [34]:
ts = collect(ts)
y_biexp = (total_area - myelin_area) .* exp.(-ts*btparams.R2_lp) .+ myelin_area .* exp.(-ts*btparams.R2_sp);

UndefVarError: UndefVarError: ts not defined

In [35]:
mxcall(:figure, 0)
mxcall(:plot, 0, collect(ts), norm.(Stotal))
mxcall(:hold, 0, "on")
mxcall(:plot, 0, collect(ts), y_biexp)

UndefVarError: UndefVarError: ts not defined

In [ ]:
# prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), u0, tspan, (Amap,));

In [ ]:
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=100); # penelope: 17.42s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=50); # penelope: 30.09s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=10); # penelope: 103.5s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-8, norm=expmv_norm); # penelope: 53.2s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-6, norm=expmv_norm); # penelope: 44.4s

In [ ]:
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-8, alg_hints=:stiff); # penelope: 90.21s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff); # penelope: 33.44s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:BCG); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 53.66s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:TFQMR); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 18.99s but low accuracy

In [ ]:
#prob_Ku = ODEProblem(K_mul_u!, u0, tspan, (K,), mass_matrix=M);
#@time sol_Ku = solve(prob_Ku, Rosenbrock23(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF
#@time sol_Ku = solve(prob_Ku, Rodas4(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF

In [ ]:
# @show norm(sol.u[end] - u)/maximum(abs,u);
# @show maximum(sol.u[end] - u)/maximum(abs,u);

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
vtk_grid("bloch_torrey_equation", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*